In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
item_name = ['Power Cord', 'Phone Charger', 'Ear Buds', 'Mouse' ,'Keyboard', 'Milk', 'Eggs', 'Cereal', 'Shrimp',
             'Noodles', 'Steak', 'King Crab', 'Tape', 'Glue', 'Nails', 'Bracket', 'Brush', 'Paint']

- train_data : 2015 1분기 ~ 2021 4분기까지 data (industry data 기준 feature)

- label_data 와 label_plus : 2015 2분기 ~ 2022년 1분기까지 data (retail sale data 'Sale')

(데이터를 따로 저장해둬서, 본 코드에서 합쳐서 사용)

- test_data : 2022년 1분기 data (industry data 기준)

> "직전분기 industry data 기준으로 다음 분기 retail sale 예측하기"

> 카테고리별로 따로 모델 학습 진행 후, allocation 결과 예측하기

In [254]:
X_category = ['Ear Buds', 'Mouse', 'King Crab','Mouse', 'Eggs']
Y_category_1 = ['Bracket','Paint']
Y_category_2 = ['Ear Buds', 'Mouse', 'King Crab', 'Mouse', 'Eggs']
# 다른 item들은 따로 학습 진행

# category 별 item의 정보를 저장하기 위한 dataframe 만들기
train= pd.DataFrame(columns=['year', 'quarter', 'item', 'store', 'inventory_value',
       'Fre_state_category', 'Fre_all_state_category', 'Fre_sales',
       'Fre_state_item', 'Fre_all_state_item', 'Fre_store_category',
       'Fre_store_item', 'Profit_store_cate', 'Profit_store_item', 'urban',
       'region', 'type'])

label = pd.DataFrame(columns=['year', 'quarter', 'item', 'store', 'sale'])

test = pd.DataFrame(columns=['year', 'quarter', 'item', 'store', 'inventory_value',
       'Fre_state_category', 'Fre_all_state_category', 'Fre_sales',
       'Fre_state_item', 'Fre_all_state_item', 'Fre_store_category',
       'Fre_store_item', 'Profit_store_cate', 'Profit_store_item', 'urban',
       'region', 'type'])

In [ ]:
for iname in X_category:
    train_data = pd.read_csv('../input/item-info/'+iname+'_quarter_total_sale.csv',encoding = 'cp949')
    label_data = pd.read_csv('../input/item-info/'+iname+'_quarter_label.csv',encoding = 'cp949')
    label_plus = pd.read_csv('../input/testingset/test_'+iname+'_quarter_label.csv',encoding = 'cp949')
    test_data = pd.read_csv('../input/testingset/test_'+iname+'_quarter_total_sale.csv', encoding ='cp949')
    
    # 학습 데이터, 테스트 데이터 만들기
    train = pd.concat([train,train_data])
    label_set = pd.concat([label_data,label_plus])
    label = pd.concat([label,label_set])
    test = pd.concat([test,test_data])

In [ ]:
# 합치는 과정에서 생긴 index 초기화 및 index column 삭제
train_data = train.reset_index()
train_data = train_data.drop(['index'],axis=1)
label_set = label.reset_index()
label_set = label_set.drop(['index'],axis=1)
test_data = test.reset_index()
test_data = test_data.drop(['index'],axis=1)

In [ ]:
label_del = label_set[label_set['year']==2015] # abc_analysis와 같이 다음분기 예측이므로, 2015 1분기 label 삭제
label_d= label_del[label_del['quarter']==1]
sample_label = label_set.drop(label_d.index)
    
sort_train = train_data.sort_values(['year','quarter','item','store'])
sort_test = test_data.sort_values(['year','quarter','item','store'])

sample_train_data = sort_train.reset_index()
sample_label_data = sample_label.reset_index()
sample_test_data = sort_test.reset_index()

sample_train_data = sample_train_data.drop(['index'],axis=1)
sample_test_data = sample_test_data.drop(['index'],axis=1)
sample_label_data = sample_label_data.drop(['index'],axis=1)

sample_label_data = sample_label_data.sort_values(['year','quarter','item','store'])

label_column = sample_label_data['sale']

from sklearn.preprocessing import LabelEncoder 
encoder = LabelEncoder() # year label encoder
sample_train_data['year'] = encoder.fit_transform(sample_train_data['year'])
sample_test_data['year']= 7

name_encoder = LabelEncoder() # category별 item들이 모여있으므로, item column -> label encoder 진행
name_encoder.fit(sample_train_data['item'])
sample_train_data['item'] = name_encoder.transform(sample_train_data['item'])

sample_train_data = sample_train_data.drop(['store'],axis=1)    
sample_test_data = sample_test_data.drop(['store'],axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() # 데이터 scale 맞추기
sample_train_data= scaler.fit_transform(sample_train_data)


from sklearn.linear_model import LinearRegression # 모델학습 진행
model = LinearRegression() 
model.fit(sample_train_data, label_column)

In [ ]:
for iname in X_category: # 카테고리 내 item 들로 한번에 학습한 모델 기반으로 각 item allocation 예측
    
    test_data = pd.read_csv('../input/testingset/test_'+iname+'_quarter_total_sale.csv', encoding ='cp949') #2022년 1분기 해당 item에 대한 데이터
    
    sort_test = test_data.sort_values(['year','quarter','store']) # 학습했던 data와 같게 sorting
    sample_test_data = sort_test.reset_index()
    sample_test_data = sample_test_data.drop(['index'],axis=1)
    
    sample_test_data['year']= 7 #encoder 진행
    sample_test_data['item'] = name_encoder.transform(sample_test_data['item']) 
    sample_test_data = sample_test_data.drop(['store'],axis=1)
    sample_test_data= scaler.transform(sample_test_data) # scaling 진행
    
    y_pred=model.predict(sample_test_data) # 예측결과
    
    result_file = pd.read_csv('../input/submission/sampling_file.csv', encoding ='cp949') #정답 파일 불러오기

    rfilter = result_file['Item'] == iname # item에 대한 부분만 추출
    r_frame = result_file[rfilter]
    result = r_frame.reset_index()
    result = result.drop(['index'],axis=1)
    result = result.sort_values(['Store'])

    s_predict= np.around(y_pred) # 예측결과가 너무 작은 경우 존재 -> 정수로 변환

    result['Allocation'] = s_predict

    result.to_csv(iname+'_allocation.csv',index=False)

In [ ]:
example_test = pd.DataFrame(columns=['Allocation','Store','Item'])

for iname in item_name:
    r_item = pd.read_csv('./'+iname+'_allocation.csv',encoding = 'cp949') # 각 item에 대한 allocation 결과 불러오기
    r_item.loc[r_item['Allocation'] <0 , 'Allocation'] = 0 # 음수인 경우, 0으로 변환
    total = r_item['Allocation'].sum() # 비율로 변환하기 위한 allocation 총량 계산
    r_item['Allocation'] = r_item['Allocation']/total * 280 # 각 item 동일한 양 곱해 allocation 계산
    r_item['Allocation']= np.around(r_item['Allocation']) #정수로 변환
    example_test = pd.concat([example_test, r_item], ignore_index=True) # 모든 item allocation 결과 합치기

In [ ]:
print(example_test['Allocation'].sum()) # allocation 총량 확인하는 과정

In [281]:
example_test.to_csv('xyz_allocation_file.csv',index=False) #최종 결과 csv 내보내기